# 微妙的字符串

In [1]:
a = 'some_thing'
b = 'some'+'_'+'thing'
id(a),id(b)

(1957716471920, 1957716471920)

In [2]:
a = 'wtf'
b = 'wtf'
a is b

True

In [3]:
a = 'wtf!'
b = 'wtf!'
a is b

False

In [4]:
a,b = 'wtf!','wtf!'
a is b

True

In [7]:
'a'*20 is 'aaaaaaaaaaaaaaaaaaaa','a'*21 is 'aaaaaaaaaaaaaaaaaaaaa'

(True, False)

 Cpython 在编译优化时, 某些情况下会尝试使用已经存在的不可变对象,成为字符串驻留
 发生驻留之后, 许多变量可能指向内存中的相同字符串对象
 所有长度为 0 和长度为 1 的字符串都被驻留.

字符串在编译时被实现 ('wtf' 将被驻留, 但是 ''.join(['w', 't', 'f'] 将不会被驻留)
字符串中只包含字母，数字或下划线时将会驻留. 所以 'wtf!' 由于包含 ! 而未被驻留。

当在同一行将 a 和 b 的值设置为 "wtf!" 的时候, Python 解释器会创建一个新对象, 然后同时引用第二个变量.

常量折叠(constant folding) 是 Python 中的一种 窥孔优化(peephole optimization) 技术. 这意味着在编译时表达式 'a'*20 会被替换为 'aaaaaaaaaaaaaaaaaaaa' 以减少运行时的时钟周期. 只有长度小于 20 的字符串才会发生常量折叠. 

In [25]:
a = 1
b = 1
a is b,id(a) == id(b)

(True, True)

is 是比较对象是否相同(is 表示对象标识符即 object identity)，即用 id() 函数查看的地址是否相同，如果相同则返回 True，如果不同则返回 False。is 不能被重载。

== 是比较两个对象的值是否相等，此操作符内部调用的是 ___eq__() 方法。所以 a==b 等效于a.___eq__(b)，所以 = 可以被重载

# 是时候来点蛋糕了!

In [16]:
some_dict = {}
some_dict[5.5] = 'ruby'
some_dict[5.0] = 'javascript'
some_dict[5] = 'python'
print(some_dict[5.0])

python


In [18]:
5 == 5.0,hash(5) == hash(5.0)

(True, True)

Python 字典通过检查键值是否相等和比较哈希值来确定两个键是否相同.
具有相同值的不可变对象在Python中始终具有相同的哈希值

# 本质上,我们都一样

In [43]:
class WTF:
    pass
print(WTF() == WTF(),WTF() is WTF())
print(hash(WTF()) == hash(WTF()))
print(id(WTF()) == id(WTF()))

False False
True
True


当调用 id 函数时, Python 创建了一个 WTF 类的对象并传给 id 函数. 然后 id 函数获取其id值 (也就是内存地址), 然后丢弃该对象. 该对象就被销毁了.

当我们连续两次进行这个操作时, Python会将相同的内存地址分配给第二个对象. 因为 (在CPython中) id 函数使用对象的内存地址作为对象的id值, 所以两个对象的id值是相同的.

In [46]:
print(id(id(WTF())) == id(id(WTF()))) #无论多少个ID都是True 原因就在上面
#虽然id(id(WTF())) == id(id(WTF())) 但是id(WTF()) is id(WTF()) 返回True
#原因就是id这个函数调用的过程特殊性
print(id(WTF()) is id(WTF())) 

True
False


In [51]:
class WTF(object):
    def __init__(self): 
        print("I")
    def __del__(self): 
        print("D")

In [53]:
WTF() is WTF() #这时是两个对象一起创建，然后一起销毁，所以id不一样

I
I
D
D


False

In [54]:
id(WTF()) == id(WTF()) #这时候先创建一个销毁，然后再创建。对象销毁的顺序是造成所有不同之处的原因.

I
D
I
D


True

# 为什么？

In [55]:
some_string = "wtf"
some_dict = {}
for i, some_dict[i] in enumerate(some_string):
    pass

In [56]:
some_dict

{0: 'w', 1: 't', 2: 'f'}

In [57]:
for i in range(4):
    print(i)
    i = 10

0
1
2
3


# 列表副本

In [60]:
list1 = [1,2,3,4,5]
list2 = list1
list2[0] = 6
print(list1,list2)

[6, 2, 3, 4, 5] [6, 2, 3, 4, 5]


In [61]:
list1 = [1,2,3,4,5]
list2 = list1[:]
list2[0] = 6
print(list1,list2)

[1, 2, 3, 4, 5] [6, 2, 3, 4, 5]


# 执行时机差异

In [68]:
array = [1, 8, 15]
g = (x for x in array if array.count(x) > 0)
array = [2, 8, 22]
print(list(g))

[8]


在生成器表达式中, in 子句在声明时执行, 而条件子句则是在运行时执行.
所以在运行前, array 已经被重新赋值为 [2, 8, 22], 因此对于之前的 1, 8 和 15, 只有 count(8) 的结果是大于 0 的, 所以生成器只会生成 8.

In [67]:
array_1 = [1,2,3,4]
g1 = (x for x in array_1)
array_1 = [1,2,3,4,5]

array_2 = [1,2,3,4]
g2 = (x for x in array_2)
array_2[:] = [1,2,3,4,5]
print(list(g1))
print(list(g2))

[1, 2, 3, 4]
[1, 2, 3, 4, 5]


第二部分中 g1 和 g2 的输出差异则是由于变量 array_1 和 array_2 被重新赋值的方式导致的.

在第一种情况下, array_1 被绑定到新对象 [1,2,3,4,5], 因为 in 子句是在声明时被执行的， 所以它仍然引用旧对象 [1,2,3,4](并没有被销毁).

在第二种情况下, 对 array_2 的切片赋值将相同的旧对象 [1,2,3,4] 原地更新为 [1,2,3,4,5]. 因此 g2 和 array_2 仍然引用同一个对象(这个对象现在已经更新为 [1,2,3,4,5]).

# 出人意料的is

In [69]:
a = 256
b = 256
a is b

True

In [70]:
a = 257 
b = 257  #256 是一个已经存在的对象, 而 257 不是
#当你启动Python 的时候, -5 到 256 的数值就已经被分配好了.
#这些数字因为经常使用所以适合被提前准备好
a is b

False

In [77]:
a,b = 257,257 #当 a 和 b 在同一行中使用相同的值初始化时，会指向同一个对象.
print(a is b)
print(id(a),id(b))

True
1957717387056 1957717387056


In [72]:
[] == []

True

In [74]:
[] is [] #两个空列表位于不同的内存地址

False

# 一蹴即至!

In [80]:
row = [""] * 3
board = [row] * 3
board

[['', '', ''], ['', '', ''], ['', '', '']]

In [81]:
board[0][0] = 'X'
board #这是因为之前对row做乘法导致的

[['X', '', ''], ['X', '', ''], ['X', '', '']]

In [83]:
#如何避免这种情况？
board = [['']*3 for _ in range(3)]
board[0][0] = 'X'
board

[['X', '', ''], ['', '', ''], ['', '', '']]